In [1]:
import math
# 皮尔逊相关度
def corr(X, Y):
    sumX = 0
    sumY = 0
    sumXY = 0
    sumX2 = 0
    sumY2 = 0
    n = len(X)
    cnt = 0
    for i in xrange(n):
        if X[i]==None or Y[i]==None:
            continue
        cnt += 1
        sumX += X[i]
        sumY += Y[i]
        sumXY += X[i] * Y[i]
        sumX2 += X[i]**2
        sumY2 += Y[i]**2
    n = cnt
    if cnt==0:
        return 0
    else:
        return (sumXY - sumX*sumY/n) / math.sqrt((sumX2-sumX**2/n)*(sumY2-sumY**2/n))

In [2]:
import MySQLdb
conn = MySQLdb.connect(host='localhost', user='root', passwd='codegeass', db='future_data', charset='utf8')
cursor = conn.cursor()

cursor.execute("select distinct date from market where date between '2014-8-1' and '2017-7-31' order by date asc")
days = [x[0] for x in cursor.fetchall()]

cursor.execute("select distinct code,name,market from market where date between '2017-1-1' and '2017-7-31'")
result = cursor.fetchall()
kinds = [x[0] for x in result]
names = [x[1] for x in result]
markets = [x[2] for x in result]


data = {}
for code in kinds:
    #print code
    cursor.execute("select date,sum(volume) from market where code='%s' and date between '2014-8-1' and '2017-7-31' group by date order by date asc" % code)
    result = {}
    data[code] = []
    for row in cursor.fetchall():
        result[row[0]] = row[1]
    for day in days:
        if day in result:
            data[code].append(result[day])
        else:
            data[code].append(None)
cursor.close()
conn.close()

In [4]:
# 统计每两个交易所商品之间的皮尔逊相关度分布
# 如果分布靠近两侧，说明交易所之间的相关度较大
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from pyecharts import Bar
bin_width = 0.05
bin_nums = 40
start = -1.0
X = ['%g~%g' % (start+(i*bin_width), start+(i+1)*bin_width) for i in range(bin_nums)]
exchange = ['SHFE','DCE','CZCE']
bar = Bar('市场间商品相关度分布', width=1000)
for i in range(len(exchange)):
    for j in range(i+1,len(exchange)):
        left = []
        for k in range(len(kinds)):
            if markets[k]==exchange[i]:
                left.append(kinds[k])
        right = []
        for k in range(len(kinds)):
            if markets[k]==exchange[j]:
                right.append(kinds[k])
        raw = []
        for l in left:
            for r in right:
                raw.append(corr(data[l],data[r]))
        Y = [0]*bin_nums
        for t in raw:
            Y[int((t-start)/bin_width)] += 1
        # 将Y转换为比例
        Y = [1.0*x/sum(Y) for x in Y]
        bar.add(exchange[i]+' '+exchange[j], X, Y, is_datazoom_show=True)
bar